In [1]:
import json

f = open('parameters.json')

data = json.load(f)

includes = data['include']
excludes = data['exclude']

if len(includes) > 0:
    print("there are includes")
elif len(excludes) > 0:
    print("there are excludes")
else:
    print("There are no parameters given")

there are excludes


In [3]:
import requests

s = requests.Session()
s.headers.update({"platform":"pc", "language":"en"})

all_items = s.get("https://api.warframe.market/v1/items").json()
items = []

for index, possible_item in enumerate(all_items["payload"]["items"]):
    for string in excludes:
        if string in possible_item["url_name"]:
            all_items["payload"]["items"].pop(index)
        else:
            items.append(possible_item["url_name"])

print(len(all_items["payload"]["items"]))
for i in items:
    print(i)

2664
secura_dual_cestra
creeping_bullseye
mutalist_alad_v_assassinate_key
irradiating_disarm
antimatter_absorb
arcane_barrier
arcane_ice
telos_boltor
frost_prime_set
catalyzer_link
targeting_subsystem
strun_wraith_receiver
tectonic_fracture
mantis_set
acid_shells
nightwatch_napalm
quasar_drill
comet_blast
nebula_bore
kaszas_blade
velocitus_receiver
fluctus_barrel
fluctus_limbs
fluctus_stock
decurion_barrel
hydroid_prime_set
cleanse_corrupted
kappa_beacon
star_crimzian
virtuos_strike
hunter_adrenaline
viper_wraith
orokin_derelict_plaza_scene
grineer_shipyards_manufactory_scene
infested_ship_bridge_scene
lua_nursery_scene
corpus_gas_city_conduit_scene
grineer_settlement_reactor_scene
corpus_ice_planet_wreckage_scene
eidolon_madurai_lens
eidolon_vazarin_lens
eidolon_unairu_lens
zephyr_prime_chassis
mycona_colony_scene
pax_soar
redeemer_prime_blade
akjagara_prime_receiver
kitgun_riven_mod_(veiled)
spectrosiphon
wolf_sledge_head
wolf_sledge_set
orb_vallis_scene
zhuge_prime_grip
freeze_force

In [ ]:
import statistics
import time
import csv

# write out results to sellPrices.csv
f = open('sellPrices.csv', 'w')
writer = csv.writer(f)

print("total items : " + str(len(all_items["payload"]["items"])))

start_time = time.time();

for index, item in enumerate(all_items["payload"]["items"]):

    url_name = item["url_name"]
    print(str(index) + "/" + str(len(all_items["payload"]["items"])) + " " + url_name)
    

    sellRequest = s.get("https://api.warframe.market/v1/items/" + url_name + "/orders").json()
    all_online_sell_prices = []

    for sItem in sellRequest["payload"]["orders"]:
        user = sItem["user"]
        if(sItem["order_type"] == "sell"):
            # ToDo : accept orders for players who have been offline, but for less than 12hr
            if user["status"] == "online" or user["status"] == "ingame":
                all_online_sell_prices.append(sItem["platinum"])

    # only add to table if item is available
    if len(all_online_sell_prices) > 0:
        mean = statistics.mean(all_online_sell_prices)
        median = statistics.median(all_online_sell_prices)

        data = [url_name, mean, median]
        writer.writerow(data)

    # Warframe.market API has a limit of 3 requests per second
    time.sleep(0.34)

f.close()
print("Total time elapsed: " + str(time.time() - start_time) + "s") 

In [ ]:
import statistics
import time
import csv

# write out results to buyPrices.csv
f = open('buyPrices.csv', 'w')
writer = csv.writer(f)

print("total items : " + str(len(items["payload"]["items"])))

start_time = time.time();

for index, item in enumerate(items["payload"]["items"]):

    print(str(index) + "/" + str(len(items["payload"]["items"])) + " " + url_name)
    url_name = item["url_name"]

    sellRequest = s.get("https://api.warframe.market/v1/items/" + url_name + "/orders").json()
    all_online_buy_prices = []

    for sItem in sellRequest["payload"]["orders"]:
        user = sItem["user"]
        if(sItem["order_type"] == "buy"):
            # ToDo : accept orders for players who have been offline, but for less than 12hr
            if user["status"] == "online" or user["status"] == "ingame":
                all_online_buy_prices.append(sItem["platinum"])

    # only add to table if item is available
    if len(all_online_buy_prices) > 0:
        mean = statistics.mean(all_online_buy_prices)
        median = statistics.median(all_online_buy_prices)

        data = [url_name, mean, median]
        writer.writerow(data)

    # Warframe.market API has a limit of 3 requests per second
    time.sleep(0.34)

f.close()
print("Total time elapsed: " + str(time.time() - start_time) + "s") 